In [1]:
# Install dependencies
!pip install gradio transformers torch pdfplumber python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade gradio

In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 133.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [ ]:
# ---- Imports ----
import os
import gradio as gr
import pdfplumber
import docx
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login  # Import the login function

# ---- Load Mistral-7B Language Model ----
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Login to Hugging Face Hub using your token
# Replace 'YOUR_HUGGING_FACE_TOKEN' with your actual token
login(token='your token of hugging face account')

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# ---- Resume Text Extraction ----
def extract_text_from_resume(resume_file):
    try:
        if resume_file.name.endswith('.pdf'):
            with pdfplumber.open(resume_file) as pdf:
                text = "\n".join(page.extract_text() or "" for page in pdf.pages)
        elif resume_file.name.endswith('.docx'):
            doc = docx.Document(resume_file)
            text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
        else:
            return "Unsupported file format"
        return text
    except Exception as e:
        print(f"Error extracting text: {e}")
        return ""

# ---- LinkedIn URL Extraction ----
def extract_linkedin_url(text):
    match = re.search(r'https?://(www\.)?linkedin\.com/in/[A-Za-z0-9\-_]+', text)
    return match.group(0) if match else ""

# ---- ATS Score + Skill Extraction ----
def extract_skills_and_score(text):
    skills = ['python', 'machine learning', 'data analysis', 'tensorflow', 'flask', 'django',
              'sql', 'pandas', 'numpy', 'keras', 'react', 'java', 'c++', 'git', 'linux']
    text_lower = text.lower()
    matched_skills = [skill for skill in skills if skill in text_lower]
    score = (len(matched_skills) / len(skills)) * 100
    return matched_skills, f"{score:.2f}% match for ATS keywords."

# ---- AI Interview Question Generator ----
def generate_questions(text, skills, num_questions=5):
    try:
        if not text.strip() or not skills:
            return "Insufficient data to generate skill-based questions."

        skills_list = ", ".join(skills)
        resume_snippet = " ".join(text.split()[:100])

        prompt = (
            f"You are an AI interviewer. A candidate has submitted the following resume snippet:\n"
            f"\"{resume_snippet}\"\n\n"
            f"The candidate claims to have skills in: {skills_list}.\n"
            f"Generate {num_questions} technical interview questions that test their understanding of these skills."
        )

        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        output = model.generate(
            **inputs,
            max_length=512,
            do_sample=True,
            top_k=50,
            temperature=0.8,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

        result = tokenizer.decode(output[0], skip_special_tokens=True)
        result = result.split(prompt)[-1].strip()
        return result
    except Exception as e:
        print(f"Error generating questions: {e}")
        return "Error generating questions"

# ---- Main Resume Processing ----
def process_resume(resume_file):
    try:
        text = extract_text_from_resume(resume_file)
        if not text:
            return "Error extracting resume text", "Error", "Error"

        linkedin_url = extract_linkedin_url(text)
        linkedin_status = "✅ Found" if linkedin_url else "❌ Not Found"

        skills, ats_score = extract_skills_and_score(text)
        generated_questions = generate_questions(text, skills)

        return linkedin_status, ats_score, generated_questions
    except Exception as e:
        print(f"Error processing resume: {e}")
        return "Error", "Error", "Error"

# ---- Gradio UI ----
with gr.Blocks() as interface:
    gr.Markdown("## 🧠 AI Resume Fraud Detection System")
    with gr.Row():
        resume_input = gr.File(label="Upload Resume (.pdf or .docx)")
        submit_btn = gr.Button("Analyze Resume")

    linkedin_output = gr.Textbox(label="LinkedIn Account Status")
    ats_output = gr.Textbox(label="ATS Score")
    questions_output = gr.Textbox(label="AI-Generated Interview Questions", lines=8)

    submit_btn.click(fn=process_resume, inputs=resume_input, outputs=[
        linkedin_output,
        ats_output,
        questions_output
    ])

interface.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e0bc9e512cb901d7cc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
